In [1]:
from pathlib import Path
from src.blocks import BlockAnalyzer, Block, block_decomposition, Decomposer

In [2]:
path_blocks = Path("out/blocks/Cluster_10644.json")
list_blocks = BlockAnalyzer()._load_list_blocks(path_blocks)

In [3]:
list_blocks

[Block(K=(1, 2), i=0, j=75, label='CACGATTCTCCCTTTGAGTTGATGAGGTTTCAGGGAAAAGGATAGCTGATTCTCCGCTTTTGCAAGTATGAAAGGC'),
 Block(K=(0, 1, 2), i=0, j=15, label='CACGATTCTCCCTTTG'),
 Block(K=(0, 1), i=17, j=81, label='GTTGATGAGGTTTCAGGGAAAAGGATAGCTGATTCTCCGCTTTTGCAAGTATGAAAGGCGAAAAA'),
 Block(K=(0, 1, 2), i=17, j=75, label='GTTGATGAGGTTTCAGGGAAAAGGATAGCTGATTCTCCGCTTTTGCAAGTATGAAAGGC'),
 Block(K=(0, 1, 2), i=77, j=81, label='AAAAA')]

In [4]:
decomposer=Decomposer()
new_blocks = decomposer(list_blocks)

In [5]:
len(new_blocks), len(list_blocks)

(9, 5)

In [6]:
import dataclasses
dataclasses.astuple(new_blocks[0])

((1, 2),
 16,
 75,
 'AGTTGATGAGGTTTCAGGGAAAAGGATAGCTGATTCTCCGCTTTTGCAAGTATGAAAGGC')

In [7]:
from collections import defaultdict
d = defaultdict(int)
for block in new_blocks:
    d[ block ] +=1
d

defaultdict(int,
            {Block(K=(1, 2), i=16, j=75, label='AGTTGATGAGGTTTCAGGGAAAAGGATAGCTGATTCTCCGCTTTTGCAAGTATGAAAGGC'): 1,
             Block(K=(0, 1, 2), i=77, j=81, label='AAAAA'): 1,
             Block(K=(0, 1, 2), i=0, j=15, label='CACGATTCTCCCTTTG'): 1,
             Block(K=(0, 1), i=17, j=81, label='GTTGATGAGGTTTCAGGGAAAAGGATAGCTGATTCTCCGCTTTTGCAAGTATGAAAGGCGAAAAA'): 1,
             Block(K=(0, 1), i=17, j=76, label='GTTGATGAGGTTTCAGGGAAAAGGATAGCTGATTCTCCGCTTTTGCAAGTATGAAAGGCG'): 1,
             Block(K=(1, 2), i=0, j=75, label='CACGATTCTCCCTTTGAGTTGATGAGGTTTCAGGGAAAAGGATAGCTGATTCTCCGCTTTTGCAAGTATGAAAGGC'): 1,
             Block(K=(0, 1, 2), i=17, j=75, label='GTTGATGAGGTTTCAGGGAAAAGGATAGCTGATTCTCCGCTTTTGCAAGTATGAAAGGC'): 1,
             Block(K=(0, 1), i=76, j=81, label='GAAAAA'): 1,
             Block(K=(1, 2), i=0, j=16, label='CACGATTCTCCCTTTGA'): 1})

In [8]:
block.to_positional_string()

PositionalString(b='CACGATTCTCCCTTTGA', i=0, j=16)

___
## Block decomposition
Given a list of blocks, decompose intersections 